# Using IBM Cloud SQL Query to ETL Weather Data

<div class="pull-left"><left><img style="float: right;" src="http://developer.ibm.com/clouddataservices/wp-content/uploads/sites/85/2018/01/ibm-cloud-object-storage-logo-small.png" width="100" margin=50></left></div>
<div style="text-align:center">
IBM Cloud SQL Query is IBM's serverless SQL service on data in Cloud Object Storage. It allows to run ANSI SQL on Parquet, CSV, JSON, ORC and AVRO data sets. You can use it to run your analytic queries, and you can use it to conduct complex transformations and write the result in any desired data format, partitioning and layout. SQL Query is based on Apache Spark SQL as the query engine in the background. This means you do not have to provision any Apache Spark instance or service. A simple Python client (like the IBM Watson Studio Notebook) is sufficient.<br><br></div>
This notebook is meant to be a generic starter to use the SQL Query API in order to run SQL statements in a programmatic way. It uses the <a href="https://github.com/IBM-Cloud/sql-query-clients/tree/master/Python" target="_blank" rel="noopener noreferrer">ibmcloudsql</a> Python library for this purpose. The notebook also demonstrates how you can combine SQL Query with visualization libraries such as PixieDust. The notebook has been verified to work with Python 3.5. As mentioned above it does not require a Spark service bound to the notebook.

## Table of contents
1. [Setup libraries](#setup)<br>
2. [Configure SQL Query](#configure)<br>
    2.1 [Using the project bucket](#projectbucket)<br>
    2.2 [Setting SQL Query parameters](#parameters)<br>
3. [Your SQL](#sql)<br>
4. [Running Your SQL Statement](#run)<br>

### <a id="setup"></a> 1. Setup libraries

Run the following cell at least once in your notebook environment in order to install required packages, such as the SQL Query client library:

In [1]:
!pip install --user ibmcloudsql

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [2]:
import ibmcloudsql
import pandas as pd

### <a id="configure"></a> 2. Configure SQL Query
1. You need an **API key** for an IBM cloud identity that has access to your Cloud Object Storage bucket for writing SQL results and to your SQL Query instance. To create API keys log on to the IBM Cloud console and go to <a href="https://console.bluemix.net/iam/#/apikeys" target="_blank">Manage->Security->Platform API Keys</a>, click the `Create` button, give the key a custom name and click `Create`. In the next dialog click `Show` and copy the key to your clipboard and paste it below in this notebook.
2. You need the **instance CRN** for the SQL Query instance. You can find it in the <a href="https://console.bluemix.net/dashboard/apps" target="_blank">IBM Cloud console dashboard</a>. Make sure you have `All Resources` selected as resource group. In the section `Services` you can see your instances of SQL Query and Cloud Object Storage. Select the instance of SQL Query that you want to use. In the SQL Query dashboard page that opens up you find a section titled **REST API** with a button labelled **Instance CRN**. Click the button to copy the CRN into your clipboard and paste it here into the notebook. If you don't have an SQL Query instance created yet, <a href="https://console.bluemix.net/catalog/services/sql-query" target="_blank">create one</a> first.
3. You need to specify the location on Cloud Object Storage where your **query results** should be written. This comprises three parts of information that you can find in the Cloud Object Storage UI for your instance in the IBM Cloud console. You need to provide it as a **URL** using the format `cos://<endpoint>/<bucket>/[<prefix>]`. You have the option to use the cloud object storage **bucket that is associated with your project**. In this case, execute the following section before you proceed.  
<br/>
For more background information, check out the SQL Query <a href="https://console.bluemix.net/docs/services/sql-query/getting-started.html#getting-started-tutorial" target="_blank">documentation</a>.

In [3]:
# @hiddencell 
# see e.g. https://console.bluemix.net/docs/tutorials/smart-data-lake.html#build-a-data-lake-using-object-storage for similar steps

# API Key
apikey = '<apikey>'

# SQL Query CRN
instnacecrn = '<crn>'

#COS endpoint
sql_cos_endpoint = 'cos://eu-de/weather-data-demo'

#### <a id="projectbucket"></a> 2.1 Using the project bucket
**Only** follow the instructions in this section when you want to write your SQL query results to the bucket that has been created for the project for which you have created this notebook. In any other case proceed directly with section **2.2**.
<br><br>
__Inserting the project token__:  
Click the `More` option in the toolbar above (the three stacked dots) and select `Insert project token`.
 * If you haven't created an access token for this project before, you will see a dialog that asks you to create one first. Follow the link to open your project settings, scroll down to `Access tokens` and click `New token`. Give the token a custom name and make sure you select `Editor` as `Access role for project`. After you created your access token you can come back to this notebook, select the empty cell below and again select `Insert project token` from the toolbar at the top.
[//]: # 

Leave that cell content as inserted and run the cell. Then then proceed with the following cell below:

In [4]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='<project-id>', project_access_token='<project token>')
pc = project.project_context

In [5]:
cos_bucket = project.get_metadata()['entity']['storage']['properties']
targeturl="cos://" + cos_bucket['bucket_region'] + "/" + cos_bucket['bucket_name'] + "/"

#### <a id="parameters"></a> 2.2 Setting the SQL Query parameters

In [6]:
sqlClient = ibmcloudsql.SQLQuery(apikey, instnacecrn, client_info='SQL Query Starter Notebook')
sqlClient.logon()
print('\nYour SQL Query web console link:\n')
sqlClient.sql_ui_link()


Your SQL Query web console link:

https://sql.ng.bluemix.net/sqlquery/?instance_crn=crn:v1:bluemix:public:sql-query:us-south:a/e97a8c01ac694e308ef3ad7795e48756:b9c3fd30-b752-4a4d-a7a7-51e2ec0d7c1f::


### <a id="sql"></a> 3. Your SQL
To author your own SQL query, use the interactive SQL Query web console (**link above**) of your SQL Query service instance.

In [7]:
sql = "WITH wd AS (SELECT day_ind, CAST(from_unixtime(valid_time_gmt) AS DATE) as obs_date, obs_name, temp \n" + \
          "FROM " + sql_cos_endpoint + "/* STORED AS JSON o WHERE o.class = 'observation') \n " + \
          "SELECT wd.obs_name, wd.obs_date, wd.day_ind, ROUND(MIN(wd.temp),1) AS mintemp, ROUND(AVG(wd.temp),1) AS avgtemp, ROUND(MAX(wd.temp),1) AS maxtemp \n " + \
          "FROM wd \n " + \
          "GROUP BY wd.obs_name, wd.obs_date, wd.day_ind \n " + \
          "ORDER BY wd.obs_name, wd.obs_date, wd.day_ind \n " + \
          "INTO {}result/ STORED AS CSV".format(targeturl)
print('\nYour SQL statement is:\n')
print(sql)


Your SQL statement is:

WITH wd AS (SELECT day_ind, CAST(from_unixtime(valid_time_gmt) AS DATE) as obs_date, obs_name, temp 
FROM cos://eu-de/weather-data-demo/* STORED AS JSON o WHERE o.class = 'observation') 
 SELECT wd.obs_name, wd.obs_date, wd.day_ind, ROUND(MIN(wd.temp),1) AS mintemp, ROUND(AVG(wd.temp),1) AS avgtemp, ROUND(MAX(wd.temp),1) AS maxtemp 
 FROM wd 
 GROUP BY wd.obs_name, wd.obs_date, wd.day_ind 
 ORDER BY wd.obs_name, wd.obs_date, wd.day_ind 
 INTO cos://eu-geo/weatherdatademo-donotdelete-pr-2qk5tcfdz1qs9t/result/ STORED AS CSV


### <a id="run"></a> 4. Running Your SQL Statement
The following cell submits the above SQL statement and waits for it to finish before printing a sample of the result set.

In [8]:
result_df = sqlClient.run_sql(sql)
if isinstance(result_df, str):
    print(result_df)

In [9]:
result_df.head(10)

,obs_name,obs_date,day_ind,mintemp,avgtemp,maxtemp
0,Hamburg/Finkenwerder,2019-04-28,D,8,8.9,13
1,Hamburg/Finkenwerder,2019-04-28,N,8,9.0,10
2,Hamburg/Finkenwerder,2019-04-29,D,8,11.7,17
3,Hamburg/Finkenwerder,2019-04-29,N,4,8.0,13
4,Hamburg/Finkenwerder,2019-04-30,D,4,11.9,17
5,Hamburg/Finkenwerder,2019-04-30,N,4,8.0,11
6,Hamburg/Finkenwerder,2019-05-01,D,9,9.6,11
7,Hamburg/Finkenwerder,2019-05-01,N,8,8.6,9
8,Hamburg/Finkenwerder,2019-05-02,D,8,9.9,13
9,Hamburg/Finkenwerder,2019-05-02,N,6,7.4,8


In [10]:
result_df.count()

obs_name    32
obs_date    32
day_ind     32
mintemp     32
avgtemp     32
maxtemp     32
dtype: int64

### <a id="next"></a> 8. Next steps
In this notebook you learned how you can use the `ibmcloudsql` library in a Python notebook to submit SQL queries on data in IBM Cloud Object Storage and how you can interact with the query results. If you want to automate such an SQL query execution as part of your cloud solution, you can use the <a href="https://console.bluemix.net/openwhisk/" target="_blank">IBM Cloud Functions</a> framework. There is a dedicated SQL function available that lets you set up a cloud function to run SQL statements with IBM Cloud SQL Query. You can find the documentation for doing this <a href="https://hub.docker.com/r/ibmfunctions/sqlquery/" target="_blank" rel="noopener noreferrer">here</a>.

### <a id="authors"></a>Authors

**Torsten Steinbach**, Torsten is the lead architect for IBM Cloud SQL Query. Previously he has worked as IBM architect for a series of data management products and services, including DB2, PureData for Analytics and Db2 on Cloud.

This version has been edited by **Einar Karlsen**. Commands asking for input and for formatting SQL queries have been removed and a default SQL query has been provided that aggregates weather temperatures from The Weather Company service. 

<hr>
Copyright &copy; IBM Corp. 2019. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>